In [1]:
import torch
from transformers import BertForSequenceClassification, BertTokenizer
import os
import numpy as np
from dataloader import Dataset
from model import Models


def check_safetensors_embedding_weights(model_load_dir='./model_checkpoint', model_name='krbert'):
    """
    로드된 BERT 모델의 Embedding 레이어에서 특정 단어의 가중치를 확인합니다.
    """
    try:
        # 1. Models 인스턴스 생성 및 로드
        dataset_instance = Dataset() # dataloader.py에 정의된 클래스 사용
        
        # Models 클래스는 __init__에서 dataset_instance를 받거나 내부적으로 생성합니다.
        web_model = Models(model_name=model_name, num_labels=2, dataset_instance=dataset_instance)
        
        # load_model을 호출하여 모델 가중치와 토크나이저를 로드합니다.
        # 이 과정에서 model_checkpoint 디렉토리의 model.safetensors 파일을 읽습니다.
        web_model.load_model(load_dir_path=model_load_dir)
        
        # 로드된 모델 객체와 토크나이저를 가져옵니다.
        model = web_model.model
        tokenizer = web_model.tokenizer

        print(f"✅ 모델 ({model_name.upper()}) 및 토크나이저 로드 완료.")
        print("-" * 50)
        
        # 2. 임베딩 가중치 접근
        # BERT 모델에서 단어 임베딩 레이어는 'embeddings.word_embeddings.weight'에 있습니다.
        word_embeddings = model.bert.embeddings.word_embeddings.weight
        
        # 임베딩 가중치의 크기 출력 (e.g., [Vocab_Size, Hidden_Size])
        print(f"Embedding Weight Shape: {word_embeddings.shape}")
        
        # 3. 특정 단어의 임베딩 벡터 확인
        # KR-BERT의 주요 토큰 몇 개를 확인합니다.
        
        # 확인할 단어 목록 (KR-BERT의 경우 Character/Subword 기반)
        test_words = ["한국", "사람", "여자", "남자", "루저", "폭력",  "[CLS]", "[SEP]"]
        
        print("\n--- 특정 단어의 임베딩 벡터 확인 ---")
        
        for word in test_words:
            # 토크나이저를 이용해 단어의 ID(index)를 얻습니다.
            try:
                # KR-BERT 토크나이저는 단어/문자를 ID로 변환합니다.
                token_id = tokenizer.convert_tokens_to_ids(word)
                
                # 해당 ID의 임베딩 가중치를 추출합니다. (텐서를 CPU로 이동 후 NumPy 변환)
                embedding_vector = word_embeddings[token_id].detach().cpu().numpy()
                
                # 임베딩 벡터의 처음 5개 차원만 출력
                print(f"'{word}' (ID: {token_id}): {embedding_vector[:5]}")
            
            except KeyError:
                print(f"Warning: '{word}'는 토크나이저의 어휘 목록에 없습니다.")
            except Exception as e:
                print(f"Error processing word '{word}': {e}")
                
        print("-" * 50)
        
    except FileNotFoundError:
        print(f"❌ 에러: 모델 저장 디렉토리 '{model_load_dir}'를 찾을 수 없습니다.")
        print("model.safetensors 파일이 해당 디렉토리에 있는지 확인하십시오.")
    except AttributeError as e:
        print(f"❌ 에러: 모델 로드 또는 접근 중 오류 발생. 모델 객체에 문제가 있을 수 있습니다.")
        print(f"상세 에러: {e}")
    except Exception as e:
        print(f"❌ 예상치 못한 오류 발생: {e}")

# ==========================================================
# TODO: 모델 가중치를 확인하려면 실제 모델 저장 경로를 지정해야 합니다.
model_load_directory = './model_checkpoint' 
# ==========================================================

# 함수 실행
check_safetensors_embedding_weights(model_load_dir=model_load_directory)

There are 2 GPU(s) available.
We will use the GPU: NVIDIA GeForce RTX 2080 Ti
max_length : 128
There are 2 GPU(s) available.
We will use the GPU: NVIDIA GeForce RTX 2080 Ti


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at snunlp/KR-BERT-char16424 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loading model from ./model_checkpoint
✅ 모델 (KRBERT) 및 토크나이저 로드 완료.
--------------------------------------------------
Embedding Weight Shape: torch.Size([16424, 768])

--- 특정 단어의 임베딩 벡터 확인 ---
'한국' (ID: 145): [-0.0562454   0.0108557  -0.05825476 -0.00025469  0.01125608]
'사람' (ID: 640): [-0.03931377  0.03173213 -0.00827596  0.04252414 -0.01495786]
'여자' (ID: 1011): [-0.02547332 -0.02484689 -0.04527205 -0.0606334   0.04343596]
'남자' (ID: 1029): [-0.05862902 -0.00043198 -0.02857768 -0.01635378  0.03356219]
'루저' (ID: 1): [ 0.01492126  0.04595394 -0.02212146  0.01701351 -0.01845635]
'폭력' (ID: 4550): [-0.04072202  0.04634293 -0.08216351 -0.02248642 -0.00760159]
'[CLS]' (ID: 2): [-0.01717631  0.03672351 -0.00662045 -0.01008655  0.02187421]
'[SEP]' (ID: 3): [-0.00344839  0.05116087 -0.01009995  0.00629113 -0.01234006]
--------------------------------------------------
